In [2]:
print('starting . . .')

starting . . .


In [3]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# 1. IMPORT LIBRARIES (นำเข้า PyTorch และ Tools ที่จำเป็น)

import sys
!{sys.executable} -m pip install torch torchvision torchaudio

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import os

# 2. HYPERPARAMETERS & CONFIGURATION
NUM_CLASSES = 72    # 72 คลาส (ตัวอักษรไทย)
BATCH_SIZE = 64
NUM_EPOCHS = 10
MODEL_NAME = "resnet18"
IMAGE_SIZE = 128
DATA_DIR = "C:\\Users\\HP\\Downloads\\train(1)\\data-train" # ตำแหน่งโฟลเดอร์หลัก # ตำแหน่งโฟลเดอร์หลัก

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import numpy as np
from PIL import Image, ImageOps, ImageFilter, ImageEnhance
import random
import math

# --- 0. DEFINE ALL CUSTOM AUGMENTATION CLASSES ---
# คลาสที่เพิ่มเข้ามาใหม่ 3 คลาสตามโค้ดของเพื่อน
class AddSaltPepperNoise(object):
    """เพิ่ม Salt & Pepper noise"""
    def __init__(self, amount=0.005):
        self.amount = amount

    def __call__(self, img):
        np_img = np.array(img)
        # ตรวจสอบว่าเป็นภาพสีหรือไม่
        if len(np_img.shape) == 3:
            # ทำ noise กับแต่ละ channel
            for i in range(np_img.shape[2]):
                channel = np_img[:, :, i]
                num_noise = int(self.amount * channel.size)
                # Salt
                coords_salt = [np.random.randint(0, s - 1, num_noise // 2) for s in channel.shape]
                channel[tuple(coords_salt)] = 255
                # Pepper
                coords_pepper = [np.random.randint(0, s - 1, num_noise // 2) for s in channel.shape]
                channel[tuple(coords_pepper)] = 0
        else: # ภาพ Grayscale
            num_noise = int(self.amount * np_img.size)
            # Salt
            coords_salt = [np.random.randint(0, s - 1, num_noise // 2) for s in np_img.shape]
            np_img[tuple(coords_salt)] = 255
            # Pepper
            coords_pepper = [np.random.randint(0, s - 1, num_noise // 2) for s in np_img.shape]
            np_img[tuple(coords_pepper)] = 0
        
        return Image.fromarray(np_img)


class RandomErosionDilation(object):
    """สุ่มทำ erosion/dilation เบา ๆ"""
    def __init__(self, prob=0.05):
        self.prob = prob

    def __call__(self, img):
        if random.random() < self.prob:
            # ต้องแปลงเป็น 'L' (Grayscale) ก่อนใช้ filter ประเภทนี้
            img_gray = img.convert('L')
            if random.random() < 0.5:
                processed_gray = img_gray.filter(ImageFilter.MinFilter(3))  # erosion
            else:
                processed_gray = img_gray.filter(ImageFilter.MaxFilter(3))  # dilation
            # ถ้าภาพเดิมเป็นสี ให้ merge กลับ
            if img.mode == 'RGB':
                # สร้างภาพสีจาก channel ที่ process แล้ว
                return Image.merge('RGB', (processed_gray, processed_gray, processed_gray))
            else:
                return processed_gray
        return img


class PadToSquare(object):
    """Padding ให้เป็นจตุรัสก่อน resize"""
    def __call__(self, img):
        w, h = img.size
        max_side = max(w, h)
        delta_w = max_side - w
        delta_h = max_side - h
        padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))
        # ใช้ fill=(0,0,0) สำหรับพื้นหลังสีดำ หรือ (255,255,255) สำหรับสีขาว
        # หากภาพเป็น Grayscale ต้องใช้ fill=0 หรือ fill=255
        fill_color = 0 if img.mode == 'L' else (0,0,0)
        return ImageOps.expand(img, padding, fill=fill_color)

# คลาสเดิมของคุณ
class RandomFontStyle(object):
    """สุ่มปรับให้ตัวอักษรหนาขึ้น/เอียงเล็กน้อย (จำลอง bold / italic)"""
    def __init__(self, prob=0.3, bold_factor=(1.0, 1.3), italic_angle=(-8, 8)):
        self.prob = prob
        self.bold_factor = bold_factor
        self.italic_angle = italic_angle

    def __call__(self, img):
        # (โค้ดใน __call__ เหมือนเดิม)
        if random.random() < self.prob:
            factor = random.uniform(*self.bold_factor)
            enhancer = ImageEnhance.Contrast(img)
            img = enhancer.enhance(factor)
            angle = random.uniform(*self.italic_angle)
            img = img.transform(
                img.size, Image.AFFINE,
                (1, math.tan(math.radians(angle)) / 2, 0, 0, 1, 0),
                resample=Image.BICUBIC
            )
        return img


# --- 1. DEFINE IMAGE PARAMETERS ---
IMAGE_SIZE = 128

# --- 2. DEFINE DATA TRANSFORMATIONS ---

# --- A. TRAIN TRANSFORMS (ปรับแก้ให้เหมือนเพื่อน) ---
train_transforms = transforms.Compose([
    PadToSquare(), # เปลี่ยนมาใช้ PadToSquare เพื่อรักษาสัดส่วน
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # Resize ทันทีหลัง Pad
    
    transforms.RandomRotation(10), # ลดองศาการหมุน
    transforms.RandomAffine(
        degrees=0, 
        translate=(0.05, 0.05), # ลดการเลื่อน
        scale=(0.95, 1.05),   # ลดการย่อขยาย
        shear=5               # ลดการเฉือน
    ),
    
    # เพิ่ม RandomFontStyle เข้ามา (เหมือนที่คุณมี)
    RandomFontStyle(prob=0.3),
    
    # ปรับ GaussianBlur และเอา ColorJitter ออก
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.8))
    ], p=0.3),
    
    # เพิ่ม Augmentation 2 ตัวใหม่ตามเพื่อน
    AddSaltPepperNoise(amount=0.005),
    RandomErosionDilation(prob=0.15),
    
    # ขั้นตอนสุดท้ายเหมือนเดิม
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# --- B. VALIDATION/TEST TRANSFORMS () ---
val_test_transforms = transforms.Compose([
    PadToSquare(), # เพิ่ม PadToSquare ให้เหมือน train set
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

print("✅ Data transformations are defined successfully!")
print("\n--- Training Transforms (Updated) ---")
print(train_transforms)
print("\n--- Validation/Test Transforms (Updated) ---")
print(val_test_transforms)

✅ Data transformations are defined successfully!

--- Training Transforms (Updated) ---
Compose(
    Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.05, 0.05), scale=(0.95, 1.05), shear=[-5.0, 5.0])
    RandomApply(
    p=0.3
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 0.8))
)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)

--- Validation/Test Transforms (Updated) ---
Compose(
    Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)


In [5]:
# (ส่วนที่ 1 และ 2 ของคุณสมมติว่าอยู่ข้างบน)

# --- ส่วนที่ 3 (แก้ไข) และ 4 ---

# 4. สร้าง Datasets แยกสำหรับ Train และ Validation
# วิธีนี้จะปลอดภัยและชัดเจนกว่าการ Split ทีหลัง
train_dataset_full = datasets.ImageFolder(
    DATA_DIR, 
    transform=train_transforms # ใช้ Train Transforms สำหรับข้อมูลทั้งหมดก่อน
)
val_dataset_full = datasets.ImageFolder(
    DATA_DIR, 
    transform=val_test_transforms # ใช้ Val Transforms สำหรับข้อมูลทั้งหมดก่อน
)


# 5. SPLIT DATA (แบ่งข้อมูลเป็น Train และ Validation: 80% : 20%)
TRAIN_RATIO = 0.8
train_size = int(TRAIN_RATIO * len(train_dataset_full))
val_size = len(train_dataset_full) - train_size

# ทำการ Split โดยใช้ indices ที่ได้จากการสุ่ม
indices = list(range(len(train_dataset_full)))
np.random.shuffle(indices) # สลับลำดับ indices
train_indices, val_indices = indices[:train_size], indices[train_size:]

# สร้าง Subsets จาก indices ที่แบ่งไว้
train_dataset = torch.utils.data.Subset(train_dataset_full, train_indices)
val_dataset = torch.utils.data.Subset(val_dataset_full, val_indices)


# 6. CREATE DATALOADERS
# แนะนำให้เริ่มที่ num_workers=0 ก่อนเพื่อความเสถียร
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=0 # <--- แนะนำให้เริ่มที่ 0
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=0 # <--- แนะนำให้เริ่มที่ 0
)

# --- ส่วนที่ 4 (เหมือนเดิม) ---
# ... (โค้ดสร้าง model, criterion, optimizer) ...
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- ส่วนที่ 5 (แก้ไขเรื่องการ Save โมเดล) ---
print("--- เริ่มการ Training ---")

best_val_accuracy = 0.0 # <--- เพิ่ม: ตัวแปรสำหรับเก็บ Accuracy ที่ดีที่สุด

for epoch in range(NUM_EPOCHS):
    # --- A. TRAINING PHASE ---
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # ... (โค้ด Train เหมือนเดิม) ...
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss_train = running_loss / len(train_dataset)
    
    # --- B. VALIDATION PHASE ---
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # ... (โค้ด Validation เหมือนเดิม) ...
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy_val = 100 * correct / total
    
    print(f'Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {epoch_loss_train:.4f} | Val Accuracy: {accuracy_val:.2f}%')
    
    # --- C. SAVE BEST MODEL LOGIC (เพิ่มเข้ามา) ---
    if accuracy_val > best_val_accuracy:
        best_val_accuracy = accuracy_val
        # บันทึกเฉพาะเมื่อ Val Accuracy ดีขึ้น
        torch.save(model.state_dict(), f'{MODEL_NAME}_best_model.pth')
        print(f"✅ New best model saved with accuracy: {best_val_accuracy:.2f}%")

print(f"--- Training เสร็จสมบูรณ์ โมเดลที่ดีที่สุดมี Val Accuracy: {best_val_accuracy:.2f}% ---")

--- เริ่มการ Training ---
Epoch 1/10 | Train Loss: 0.1827 | Val Accuracy: 98.14%
✅ New best model saved with accuracy: 98.14%
Epoch 2/10 | Train Loss: 0.0756 | Val Accuracy: 98.05%
Epoch 3/10 | Train Loss: 0.0607 | Val Accuracy: 97.60%
Epoch 4/10 | Train Loss: 0.0539 | Val Accuracy: 98.49%
✅ New best model saved with accuracy: 98.49%
Epoch 5/10 | Train Loss: 0.0477 | Val Accuracy: 97.85%
Epoch 6/10 | Train Loss: 0.0417 | Val Accuracy: 98.71%
✅ New best model saved with accuracy: 98.71%
Epoch 7/10 | Train Loss: 0.0428 | Val Accuracy: 98.82%
✅ New best model saved with accuracy: 98.82%
Epoch 8/10 | Train Loss: 0.0400 | Val Accuracy: 99.10%
✅ New best model saved with accuracy: 99.10%
Epoch 9/10 | Train Loss: 0.0321 | Val Accuracy: 98.62%
Epoch 10/10 | Train Loss: 0.0364 | Val Accuracy: 98.73%
--- Training เสร็จสมบูรณ์ โมเดลที่ดีที่สุดมี Val Accuracy: 99.10% ---
